<a href="https://colab.research.google.com/github/Abubakargithub/Chellenge2.0/blob/master/Copy_of_image_classified_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports here
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
import numpy as np
from torch.autograd import Variable

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'


In [0]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.Resize(255),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], 
                                                      [0.229, 0.224, 0.225])])


# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform = train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform = valid_transforms)
# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32)


In [0]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
cat_to_name


{'1': 'pink primrose',
 '10': 'globe thistle',
 '100': 'blanket flower',
 '101': 'trumpet creeper',
 '102': 'blackberry lily',
 '11': 'snapdragon',
 '12': "colt's foot",
 '13': 'king protea',
 '14': 'spear thistle',
 '15': 'yellow iris',
 '16': 'globe-flower',
 '17': 'purple coneflower',
 '18': 'peruvian lily',
 '19': 'balloon flower',
 '2': 'hard-leaved pocket orchid',
 '20': 'giant white arum lily',
 '21': 'fire lily',
 '22': 'pincushion flower',
 '23': 'fritillary',
 '24': 'red ginger',
 '25': 'grape hyacinth',
 '26': 'corn poppy',
 '27': 'prince of wales feathers',
 '28': 'stemless gentian',
 '29': 'artichoke',
 '3': 'canterbury bells',
 '30': 'sweet william',
 '31': 'carnation',
 '32': 'garden phlox',
 '33': 'love in the mist',
 '34': 'mexican aster',
 '35': 'alpine sea holly',
 '36': 'ruby-lipped cattleya',
 '37': 'cape flower',
 '38': 'great masterwort',
 '39': 'siam tulip',
 '4': 'sweet pea',
 '40': 'lenten rose',
 '41': 'barbeton daisy',
 '42': 'daffodil',
 '43': 'sword lily',

In [0]:
# TODO: Build and train your network
model = models.vgg16(pretrained = True)

In [0]:
for param in model.parameters():
    param.requires_grad = False


classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu1', nn.ReLU()),
                          ('dropout1', nn.Dropout(p=0.5)),
                          ('fc2', nn.Linear(4096, 512)),
                          ('relu2', nn.ReLU()),
                          ('dropout2', nn.Dropout(p=0.5)),
                          ('fc3', nn.Linear(512, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [0]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [0]:
# Criterion and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [0]:
def validation(model, valid_loader, criterion):
    valid_loss = 0
    accuracy = 0
    model.to('cuda')
    for images, labels in valid_loader:
        images, labels = images.to('cuda'), labels.to('cuda')
        output = model.forward(images)
        valid_loss += criterion(output, labels).item()
        
        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    return valid_loss, accuracy

In [0]:
epochs = 10
print_every = 40
steps = 0

model.to('cuda')

for e in range(epochs):
    running_loss = 0
    model.train()
    for inputs, labels in trainloader:
        steps+=1
        
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
        
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                valid_loss, accuracy = validation(model, validloader, criterion)
                
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                  "Validation Loss: {:.3f}.. ".format(valid_loss/len(validloader)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(validloader)))
            
            running_loss = 0
            
            model.train()

Epoch: 1/10..  Training Loss: 4.547..  Validation Loss: 3.420..  Validation Accuracy: 0.242
Epoch: 1/10..  Training Loss: 3.506..  Validation Loss: 2.270..  Validation Accuracy: 0.471
Epoch: 2/10..  Training Loss: 1.119..  Validation Loss: 1.673..  Validation Accuracy: 0.562
Epoch: 2/10..  Training Loss: 2.378..  Validation Loss: 1.285..  Validation Accuracy: 0.648
Epoch: 2/10..  Training Loss: 2.230..  Validation Loss: 1.120..  Validation Accuracy: 0.699
Epoch: 3/10..  Training Loss: 1.747..  Validation Loss: 0.993..  Validation Accuracy: 0.733
Epoch: 3/10..  Training Loss: 1.848..  Validation Loss: 0.834..  Validation Accuracy: 0.776
Epoch: 4/10..  Training Loss: 0.488..  Validation Loss: 0.774..  Validation Accuracy: 0.802
Epoch: 4/10..  Training Loss: 1.705..  Validation Loss: 0.766..  Validation Accuracy: 0.783
Epoch: 4/10..  Training Loss: 1.731..  Validation Loss: 0.737..  Validation Accuracy: 0.807
Epoch: 5/10..  Training Loss: 1.141..  Validation Loss: 0.668..  Validation Accu

In [0]:
# TODO: Do validation on the test set
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Do validation on the test set
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in trainloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy achieved by the network on test images is: %d%%' % (100 * correct / total))

Accuracy achieved by the network on test images is: 80%


In [0]:
# TODO: Save the checkpoint 
model.class_to_idx = train_data.class_to_idx
checkpoint = {
    'epoch' : 10,
    'arch' : 'vgg16',
    'class_to_idx': model.class_to_idx,
    'model_state_dic' : model.state_dict(),
    'optimizer_state_dic' : optimizer.state_dict(),
    'classifier' : model.classifier
}
torch.save(checkpoint, 'my_trained_model.pth')

In [0]:
from google.colab import drive
     drive.mount('/content/gdrive/file_mount=true')

IndentationError: ignored

In [0]:
drive_path='/content/gdrive/My Drive/'
file_path= ('/content/my_trained_model.pth')

In [0]:
from google.colab import files
files.download("my_trained_model.pth")

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 52434, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil